In [ ]:
# Improvement 1 from chatgpt on the same
'''
In this code, I have made the following changes:

Moved the applicant_data dictionary creation inside the tab loop, ensuring a new dictionary is created for each tab iteration.
Created a new tab_data dictionary to store the extracted data for the current tab.
Appended tab_data to the df_data DataFrame for each tab iteration.
'''
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

options = Options()
options.add_experimental_option("prefs", {
  "download.default_directory": r"C:\Users\ip 5\mike\work_related\data",
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})

browser = webdriver.Chrome(executable_path=r'C:\Users\ip 5\mike\work_related\chromedriver.exe', chrome_options=options)
browser.get('https://portal.erc.go.ke:8446/site/login')

username = browser.find_element(By.NAME, 'LoginForm[username]')
password = browser.find_element(By.NAME, 'LoginForm[password]')

username.send_keys("Kiprotich.Bii")
password.send_keys("Sup.Reem23!")
time.sleep(15)
login_button = browser.find_element(By.NAME, 'login-button')
login_button.click()

# Navigating to the electricity department menu
time.sleep(10)
actions = ActionChains(browser)
main_menu = browser.find_element(By.XPATH, '/html/body/div[2]/header/nav/div/div/div[2]/ul/li[7]')
actions.move_to_element(main_menu).perform()
sub_menu = browser.find_element(By.XPATH, '/html/body/div[2]/header/nav/div/div/div[2]/ul/li[7]/ul/li[1]')
sub_menu.click()

# Clicking the Excel Button
excel_button = browser.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div[1]/button[1]')
excel_button.click()
time.sleep(5)

# Load the Excel file
df = pd.read_excel('C:/Users/ip 5/mike/work_related/data/ERC Internal Portal.xlsx')
data = {"Links": []}
df_data = pd.DataFrame()

# Loop through the rows of the column containing the 'review?ref=' strings
for row in df.iloc[2:, 0]:
    # Split the string at the equal sign to extract the value
    my_string = 'https://portal.erc.go.ke:8446/new-licence-applications/review?ref='
    my_integer = row

    # Use an f-string to insert the integer into the string
    result = f'{my_string}{my_integer}'
    data["Links"].append(result)
    browser.get(result)
    tabs = browser.find_elements(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div[2]/div/ul/li')
    applicant_data = {}

    for tab in tabs:
        tab.click()
        time.sleep(2)
        # Define the root XPATH
        root_xpath = '/html/body/div[2]/div/div/div[2]/div/div[2]/div/div/div[2]/div/form'
        # Locate all the input elements that are children of div elements under the root XPATH
        input_elements = browser.find_elements(By.XPATH, f'{root_xpath}//div/input')
        # Define a dictionary to store the extracted data for the current tab
        tab_data = {}

        # Loop through the located input elements
        for i, input_element in enumerate(input_elements):
            # Extract the value of the current input element
            input_value = input_element.get_attribute('value')
            # Store the extracted value in the dictionary using a column name based on the index of the input element
            tab_data[f'Input {i + 1}'] = input_value

        # Append the extracted data for the current tab to the DataFrame
        df_data = pd.concat([df_data, pd.DataFrame(tab_data, index=[0])], ignore_index=True)

df_company = pd.DataFrame(data)
df_company.to_excel("output_1.xlsx", index=False)
df_data.to_excel('input_values.xlsx', index=False)

browser.quit()
